In [2]:
#kaggle titanic
import pandas as pd
import numpy as np
import os
TITANIC_PATH = os.path.join("py3env","my_env","datasets","titanic")#ファイルはosを使う

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

train_data=load_titanic_data("train.csv")
test_data=load_titanic_data("test.csv")
#csv_path=r"C:\Users\a.sakata\py3env\train.csv"
#csv2_path=r"C:\Users\a.sakata\py3env\test.csv"
#titanic=pd.read_csv(csv_path)
#titanic_test=pd.read_csv(csv2_path)
train_data.head()
train_data.info()
train_data.describe()#データチェック
train_data["Survived"].value_counts()
train_data["Pclass"].value_counts()
train_data["Embarked"].value_counts()



from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.attribute_names]
    

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
    ('select_numeric', DataFrameSelector(["Age","SibSp","Parch","Fare"])),
    ("imputer",SimpleImputer(strategy="median")),
    ("std_sclaer", StandardScaler()),
])

#カテゴリーの欠損値を捨てないこと　string用imputer
#最も大量にでた値で欠損値埋める
class MostFreqentInputer(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        self.most_frequent_=pd.Series([X[c].value_counts().
                                        index[0] for c in X],
                                      index=X.columns)
        
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

from sklearn.preprocessing import OneHotEncoder
#Pclassを数値属性パイプラインではなくて、カテゴリー属性パイプラインで処理
cat_pipeline = Pipeline([
('selecto_cat', DataFrameSelector(["Pclass","Sex","Embarked"])),
("imputer", MostFreqentInputer()),
('encoder', OneHotEncoder(sparse=False,categories='auto'))##categories='auto',
])

from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
('num_pipeline',num_pipeline),
('cat_pipeline',cat_pipeline),
])

#full_pipeline.fit(X,Y)
X_train=preprocess_pipeline.fit_transform(train_data)
print(X_train)

y_train = train_data["Survived"]

#SVMとrandomforestを分類器として利用
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_clf.fit(X_train,y_train)
X_test=preprocess_pipeline.transform(test_data)
y_pred=svm_clf.predict(X_test)

from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

from sklearn.ensemble import RandomForestClassifier

forest_clf=RandomForestClassifier(random_state=42)
forest_scores=cross_val_score(forest_clf, X_train, y_train,cv=10)
print(forest_scores.mean())

import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot([1]*10,svm_scores, ".")
plt.plot([2]*10,forest_scores,".")
plt.boxplot([svm_scores, forest_scores],labels=("SVM","RandomForest"))
plt.ylabel("Accuracy",fontsize=14)
plt.show()

from sklearn.linear_model import SGDClassifier
param_grid=[{
    "penalty":["l2","l1"],
    "learning_rate":["constant", "optimal", "invscaling", "adaptive"],
    'eta0':[0.01,0.1,0.3,0.5,0.7],
}]
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                          scoring="neg_mean_squared_error",
                           refit=True,
                           verbose=3,
                           n_jobs=-1,
                           error_score='raise'
                        )

grid_search.fit(X_train,y_train)
y_pred_SGD=grid_search.predict(X_test)
print(y_pred_SGD)
y_train_SGD=grid_search.predict(X_train)
print(accuracy_score(y_train,y_train_SGD))
#Y_model=grid_search.predict(X)
from sklearn.metrics import accuracy_score
#print(accuracy_score(Y,Y_model))
#t=grid_search.predict(x)





        

FileNotFoundError: [Errno 2] No such file or directory: 'py3env\\my_env\\datasets\\titanic\\train.csv'

In [ ]:
#関数使う
#X=titanic.drop(["Survived","Name","Cabin","Ticket","Embarked"],axis=1)
#x=titanic_test.drop(["Name","Cabin","Ticket","Embarked"],axis=1)
#Y=titanic["Survived"]
#二度手間です。　属性抽出クラスがあるので、　dropとか属性抽出変換いらない。
#X_num=X.drop(["Sex"],axis=1)
#X_num=X_num.drop(["Embarked"],axis=1)
#num_attribs=list(X_num)
#cat_attribs=["Sex"]
#属性抽出変換あるのにまだ手間しとる